In [2]:
! pip install kaggle

In [3]:
! mkdir ~/.kaggle

In [6]:
! cp kaggle.json ~/.kaggle/

In [7]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
! kaggle competitions download fake-news

 80% 37.0M/46.5M [00:00<00:00, 116MB/s]
100% 46.5M/46.5M [00:00<00:00, 126MB/s]


In [9]:
!unzip fake-news.zip

Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [10]:
import pandas as pd
df = pd.read_csv('train.csv')

In [11]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
#drop nan values
df = df.dropna()

In [13]:
x = df.drop('label',axis =1)
y = df['label']

In [59]:
print(x.shape)
print(y.shape)

(18285, 4)
(18285,)


In [15]:
#checking tensorflow version and importing this package
import tensorflow as tf
tf.__version__

'2.12.0'

In [66]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [17]:
# Vocabulary size
voc_size = 5000

#One Hot Representation

In [18]:
messages = x.copy()
messages.reset_index(inplace = True)

In [19]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package bcp47 to /root/nltk_data...
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package bllip_wsj_no_aux to /root/nltk_data...
       |   Unzipping models/bllip_wsj_no_aux.zip.
       | Downloading package book_grammars to /root/nltk_data...
       |   Un


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [20]:
from nltk.tokenize import word_tokenize
# Dataset preprocessing

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

#In the below line title is a sentence
#Remove stopwords
#Remove all characters except a-zA-Z
#Convert all characters to lower case
#Convert each word to it's stem word using PorterStemmer
corpus = [(' '.join([ps.stem(word.lower()) for word in word_tokenize(re.sub('[^a-zA-Z]',' ',title)) if word not in stopwords.words('english')])) for title in messages['title']]

In [21]:
print(corpus[0])
print(messages['title'][0])

hous dem aid we didn even see comey letter until jason chaffetz tweet it
House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It


In [29]:
#Convert to one-hot representation
one_hot_rep = [one_hot(sentence,voc_size) for sentence in corpus]
type(one_hot_rep)

list

In [23]:
#You can take any value of i(<len) here -> Output will always be True
i = 103
len(one_hot_rep[i]) == len(word_tokenize(corpus[i]))


True

In [30]:
#Now all the sentences are of diff len -> diff len of words list
#We will use padding to bring all len to same value
#We will use pad_sequences imported befor

maxi = 0
for lst in one_hot_rep:
  maxi = max(maxi,len(lst))
print(maxi)

49


In [80]:
sent_length = maxi
embedding_docs = pad_sequences(one_hot_rep,padding='pre',maxlen=sent_length)
type(embedding_docs[0])

numpy.ndarray

In [81]:
#Create model

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 49, 40)            200000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [82]:
print(type(embedding_docs))
print(type(y))

<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>


In [83]:
import numpy as np
x_final = np.array(embedding_docs)
y_final = np.array(y)

In [84]:
print(x_final.shape,y_final.shape)

(18285, 49) (18285,)


In [85]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_final,y_final,test_size=0.33,random_state=42)

In [86]:
#Model Training
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 27s 122ms/step - loss: 0.3167 - accuracy: 0.8518 - val_loss: 0.1855 - val_accuracy: 0.9239
Epoch 2/10
192/192 [==============================] - 26s 135ms/step - loss: 0.1268 - accuracy: 0.9504 - val_loss: 0.1886 - val_accuracy: 0.9244
Epoch 3/10
192/192 [==============================] - 21s 112ms/step - loss: 0.0857 - accuracy: 0.9680 - val_loss: 0.2086 - val_accuracy: 0.9226
Epoch 4/10
192/192 [==============================] - 25s 132ms/step - loss: 0.0561 - accuracy: 0.9818 - val_loss: 0.2470 - val_accuracy: 0.9216
Epoch 5/10
192/192 [==============================] - 25s 132ms/step - loss: 0.0352 - accuracy: 0.9884 - val_loss: 0.2789 - val_accuracy: 0.9210
Epoch 6/10
192/192 [==============================] - 24s 123ms/step - loss: 0.0222 - accuracy: 0.9936 - val_loss: 0.3159 - val_accuracy: 0.9155
Epoch 7/10
192/192 [==============================] - 23s 119ms/step - loss: 0.0142 - accuracy: 0.9964 - val_loss: 0.3780 - val_ac

In [88]:
#Performance
Y_pred = (model.predict(X_test) > 0.5).astype("int32")

189/189 [==============================] - 4s 17ms/step


In [89]:
from sklearn.metrics import confusion_matrix

In [90]:
confusion_matrix(Y_test,Y_pred)

array([[3106,  313],
       [ 197, 2419]])

In [91]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_pred)

0.9154929577464789